## 单进程指定GPU进行计算

在Pytorch框架下，单进程指定GPU进行计算核心语句：`device = torch.device(“cuda:gpu编号”)`
例如： `device = torch.device(“cuda:1”)`

GPU编号、显存大小、当前正在使用GPU的进程都可通过命令行语句 `nvidia-smi` 查看

此外，还可以通过 `CUDA_VISIBLE_DEVICES` 变量指定GPU

对于模型（类），使用 `.to(device)` 语句来将模型的所有参数、缓存放到指定的GPU上进行计算

注：只要显存足够，多个terminal分别执行多个进程，可以指定在同一个GPU，也可以指定在不同的GPU，即可实现多进程并行

In [ ]:
'''
Demo, 单进程指定GPU计算
'''

import torch

# 1. 检查可用的GPU数量和编号
device_count = torch.cuda.device_count()
print(f"系统中有 {device_count} 块可用的GPU。")
if device_count > 0:
    print("它们的编号是：")
    for i in range(device_count):
        print(f"GPU {i}")
else:
    print("没有可用的GPU。")

# 2. 随机生成两个可乘矩阵，并在指定的GPU上进行乘法运算
if device_count > 0:
    # 让用户输入想要使用的GPU编号
    gpu_id = int(input("请输入你想要使用的GPU编号(0到{}):".format(device_count - 1)))
    
    # 检查输入的GPU编号是否有效
    if gpu_id < 0 or gpu_id >= device_count:
        print("输入的GPU编号无效。")
    else:
        # 指定设备
        device = torch.device(f"cuda:{gpu_id}")
        
        # 在指定的GPU上创建两个随机的可乘矩阵
        matrix1 = torch.rand(3, 3, device=device)
        matrix2 = torch.rand(3, 4, device=device)
        
        print(f"在GPU {gpu_id} 上的随机矩阵1:\n{matrix1}")
        print(f"在GPU {gpu_id} 上的随机矩阵2:\n{matrix2}")
        
        # 执行矩阵乘法
        result = torch.matmul(matrix1, matrix2)
        
        print(f"矩阵乘法的结果:\n{result}")
else:
    print("由于没有可用的GPU，无法执行GPU上的矩阵乘法。")


In [ ]:
'''
Demo, 指定进程的GPU可见范围

CUDA_VISIBLE_DEVICES="1"           Only device 1 will be seen
CUDA_VISIBLE_DEVICES="0,1"         Devices 0 and 1 will be visible
CUDA_VISIBLE_DEVICES="0,1"         Same as above, quotation marks are optional
CUDA_VISIBLE_DEVICES="0,2,3"       Devices 0, 2, 3 will be visible; device 1 is masked
CUDA_VISIBLE_DEVICES=""            No GPU will be visible
'''

import os
import torch

# 设置环境变量，不使用GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# 指定设备为GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("当前使用的设备:", device)

# 在指定的GPU上创建两个可以做乘法的矩阵
matrix1 = torch.rand(2, 3, device=device)
matrix2 = torch.rand(3, 2, device=device)

# 打印生成的矩阵
print("Matrix 1:")
print(matrix1)
print("Matrix 2:")
print(matrix2)

# 进行矩阵乘法
result = torch.matmul(matrix1, matrix2)

# 打印结果
print("Result of matrix multiplication:")
print(result)

In [ ]:
'''
Demo, 把模型放到GPU上做计算
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义一个简单的神经网络模型
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(10, 5) 
        # self.to(torch.device("cuda:1")) # 定义网络时就指定设备

    # 前向传播过程
    def forward(self, x):
        x = self.fc(x)
        return F.relu(x) 

# 指定GPU设备
device = torch.device("cuda:1")

# 实例化模型
model = SimpleNet()

# 将模型移动到指定的设备
model.to(device)

# 创建一个随机数据张量，模拟输入数据
input_data = torch.randn(5, 10) 

# 将输入数据也移动到指定的设备，数据与模型在同一设备上可以提高计算效率
input_data = input_data.to(device)

# 前向传播，获取模型输出
output = model(input_data)

# 打印输出结果
print(output)

## 单进程跨GPU做计算

在深度学习和分布式计算领域，DP 通常指的是 DataParallel。DataParallel 是一种将计算任务在多个 GPU 上并行执行的方法。它在单机多卡环境中非常有用，可以在多个 GPU 上分摊工作负载，从而加快训练速度。

torch.nn.DataParallel 是 PyTorch 中的一个工具，可以让模型在多个 GPU 上并行运行。它通过将输入批次拆分成多个子批次，每个子批次发送到不同的 GPU 上，并行执行前向传播和反向传播，然后将每个 GPU 上的梯度聚合到主 GPU 上进行参数更新。

使用 DataParallel 的基本步骤
- 定义模型: 创建神经网络模型
- 包装模型: 使用 torch.nn.DataParallel 包装模型
- 将模型和数据迁移到 GPU: 使用 .to(device) 将模型和输入数据迁移到合适的设备上
- 训练模型: 按照常规方式训练模型

单机多卡训练策略
- 数据拆分，模型不拆分
- 数据不拆分，模型拆分
- 数据拆分，模型拆分

DataParallel 的局限性
- 数据并行粒度: DataParallel 进行的是数据并行操作，每个 GPU 处理一部分数据批次。由于其自动分配负载，这可能导致 GPU 利用率不均衡，尤其是在有计算负载差异的情况下
- 单节点限制: DataParallel 主要用于单节点多 GPU，即单机多卡。如果需要跨节点并行（分布式训练，多机多卡），应该考虑使用 torch.nn.parallel.DistributedDataParallel

In [2]:
'''
Demo, 单机多卡, 单进程跨GPU计算

最简单高效的策略: 数据拆分, 模型不拆分
'''

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 设置环境变量，指定使用的GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 定义设备
globalDevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义一个简单的CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 1)
        self.fc = nn.Linear(16 * 26 * 26, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 实例化模型
cnn = CNN().to(globalDevice)

# 检查GPU数量并设置DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    net = nn.DataParallel(cnn)
else:
    print("Using single GPU or CPU")
    net = cnn

# 定义数据集和数据加载器
class SimpleDataset(Dataset):
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return torch.randn(3, 28, 28), torch.tensor(1)

dataset = SimpleDataset(1000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=20)

# 定义优化器和损失函数
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# 简单的训练过程
for epoch in range(5):
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(globalDevice), labels.to(globalDevice) # 此处可指定GPU以实现手动分配负载
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Using 2 GPUs


Epoch 1, Loss: 0.0051126438193023205
Epoch 2, Loss: 0.003030891064554453
Epoch 3, Loss: 0.001992578152567148
Epoch 4, Loss: 0.0015859566628932953
Epoch 5, Loss: 0.001422481844201684


In [ ]:
'''
Pseudocode，策略一：数据拆分，模型不拆分

在这种策略中，将数据拆分成多个批次，每个批次在一个GPU上进行处理。模型不会拆分，而是复制到每个GPU上
'''

import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, Dataset  
from torch.nn.parallel import DataParallel  

# 假设我们有一个自定义的数据集和模型  
class MyDataset(Dataset):  
    # 实现__len__和__getitem__方法  
    pass  
class MyModel(nn.Module):  
    # 定义模型结构  
    pass  

# 初始化数据集和模型  
dataset = MyDataset()  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)  
model = MyModel()  

# 检查GPU数量  
device_ids = list(range(torch.cuda.device_count()))  
model = DataParallel(model, device_ids=device_ids).to(device_ids[0])  

# 定义损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# 训练循环  
for epoch in range(num_epochs):  
    for inputs, labels in dataloader:  
        inputs, labels = inputs.to(device_ids[0]), labels.to(device_ids[0])  
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()

In [ ]:
'''
Pseudocode，策略二：数据不拆分，模型拆分

在这种策略中，整个数据集在每个GPU上都会有一份副本，但模型会被拆分成多个部分，每个部分在一个GPU上运行。这种策略通常不常见，因为数据复制会消耗大量内存，而且模型拆分也可能会导致通信开销增加
'''
# 假设我们有一个可以拆分的模型（例如，具有多个子网络的模型）  
class SplitModel(nn.Module):  
    def __init__(self):  
        super(SplitModel, self).__init__()  
        self.subnet1 = nn.Sequential(...)  # 定义子网络1  
        self.subnet2 = nn.Sequential(...)  # 定义子网络2  
        # ... 其他子网络 ...  
    def forward(self, x):  
        # 前向传播逻辑，可能涉及跨多个设备的通信和数据传输  
        pass  

# 初始化模型和数据集（这里不实际拆分数据）  
model = SplitModel()  
dataset = MyDataset()  

# 将模型的每个子网络分配到一个GPU上  
model.subnet1 = model.subnet1.to('cuda:0')  
model.subnet2 = model.subnet2.to('cuda:1')  

# ... 其他子网络 ...  

# 训练循环（这里省略了数据加载和批处理，因为数据没有拆分）  
for epoch in range(num_epochs):  
    inputs, labels = ...  # 加载数据  
    inputs = inputs.to('cuda:0')  # 假设输入数据首先被送到第一个GPU上  
    optimizer.zero_grad()  
    outputs = model(inputs)  # 前向传播可能涉及跨多个GPU的通信  
    loss = criterion(outputs, labels)  
    loss.backward()  
    optimizer.step()


In [ ]:
'''
Pseudocode，策略三：数据拆分，模型拆分

在这种策略中，同时使用数据并行和模型并行。数据被拆分成多个批次，每个批次在不同的GPU上进行处理，同时模型也被拆分成多个部分，每个部分在不同的GPU上运行。这通常用于非常大的模型，单个GPU无法容纳整个模型的情况
'''

import torch  
import torch.distributed as dist  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, Dataset, DistributedSampler  
from torch.nn.parallel import DistributedDataParallel as DDP  

# 自定义数据集和模型  
class MyDataset(Dataset):  
    # 实现__len__和__getitem__方法  
    pass  
class MyModel(nn.Module):  
    # 定义模型结构，可能需要考虑如何拆分模型  
    pass  

# 初始化分布式环境  
dist.init_process_group(backend='nccl', init_method='tcp://localhost:23456', rank=0, world_size=torch.cuda.device_count())  

# 初始化数据集和模型  
dataset = MyDataset()  
sampler = DistributedSampler(dataset)  
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, sampler=sampler)  
model = MyModel()  

#拆分模型（这通常需要根据模型的具体结构来手动完成。例如，如果模型有两个主要部分，可以将它们分别放到不同的设备上  
model_part1 = model.part1.to('cuda:0')  
model_part2 = model.part2.to('cuda:1')  

# 使用DistributedDataParallel包装模型  
model = DDP(model, device_ids=[torch.cuda.current_device()])

# 定义损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# 训练循环  
for epoch in range(num_epochs):  
    for inputs, labels in dataloader:  
        inputs, labels = inputs.to(model.device), labels.to(model.device)  
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

# 销毁分布式进程组  
dist.destroy_process_group()